<a href="https://colab.research.google.com/github/Pranjal0211/heart-disease-diagnosis/blob/main/Expert_HeartDisease_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 5.4 MB/s eta 0:00:00


In [ ]:
import numpy as np
import matplotlib.pyplot as plt # Visuals
import seaborn as sns
import sklearn as skl
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import JointProbabilityDistribution
from pgmpy.estimators import MaximumLikelihoodEstimator, BayesianEstimator
from pgmpy.inference import VariableElimination

In [ ]:
heartDisease = pd.read_csv('./sample_data/heart_disease_uci_copy10.csv')
heartDisease

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,num
0,63,Male,typical angina,145,233,True,lv hypertrophy,150,False,0
1,67,Male,asymptomatic,160,286,False,lv hypertrophy,108,True,2
2,67,Male,asymptomatic,120,229,False,lv hypertrophy,129,True,1
3,37,Male,non-anginal,130,250,False,normal,187,False,0
4,41,Female,atypical angina,130,204,False,lv hypertrophy,172,False,0
5,56,Male,atypical angina,120,236,False,normal,178,False,0
6,62,Female,asymptomatic,140,268,False,lv hypertrophy,160,False,3
7,57,Female,asymptomatic,120,354,False,normal,163,True,0
8,63,Male,asymptomatic,130,254,False,lv hypertrophy,147,False,2
9,53,Male,asymptomatic,140,203,True,lv hypertrophy,155,True,1


In [ ]:
heartDisease.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,num
0,63,Male,typical angina,145,233,True,lv hypertrophy,150,False,0
1,67,Male,asymptomatic,160,286,False,lv hypertrophy,108,True,2
2,67,Male,asymptomatic,120,229,False,lv hypertrophy,129,True,1
3,37,Male,non-anginal,130,250,False,normal,187,False,0
4,41,Female,atypical angina,130,204,False,lv hypertrophy,172,False,0


In [ ]:
heartDisease.dtypes

age          int64
sex         object
cp          object
trestbps     int64
chol         int64
fbs           bool
restecg     object
thalch       int64
exang         bool
num          int64
dtype: object

In [ ]:
object_columns = heartDisease.select_dtypes(include=['bool']).columns
[print(col_name, ":", heartDisease[col].unique()) for col, col_name in zip(heartDisease[object_columns], object_columns)]

fbs : [ True False]
exang : [False  True]


[None, None]

In [ ]:
print(pd.isna(heartDisease).sum())

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
num         0
dtype: int64


In [ ]:
heartDisease = heartDisease.dropna(axis=0)
heartDisease

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,num
0,63,Male,typical angina,145,233,True,lv hypertrophy,150,False,0
1,67,Male,asymptomatic,160,286,False,lv hypertrophy,108,True,2
2,67,Male,asymptomatic,120,229,False,lv hypertrophy,129,True,1
3,37,Male,non-anginal,130,250,False,normal,187,False,0
4,41,Female,atypical angina,130,204,False,lv hypertrophy,172,False,0
5,56,Male,atypical angina,120,236,False,normal,178,False,0
6,62,Female,asymptomatic,140,268,False,lv hypertrophy,160,False,3
7,57,Female,asymptomatic,120,354,False,normal,163,True,0
8,63,Male,asymptomatic,130,254,False,lv hypertrophy,147,False,2
9,53,Male,asymptomatic,140,203,True,lv hypertrophy,155,True,1


In [ ]:
#Label encoding for boolean data.
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()
boolean_columns = heartDisease.select_dtypes(include=['bool']).columns
for col in boolean_columns:
    heartDisease[col] = label_encoder.fit_transform(heartDisease[col])
heartDisease

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,num
0,63,Male,typical angina,145,233,1,lv hypertrophy,150,0,0
1,67,Male,asymptomatic,160,286,0,lv hypertrophy,108,1,2
2,67,Male,asymptomatic,120,229,0,lv hypertrophy,129,1,1
3,37,Male,non-anginal,130,250,0,normal,187,0,0
4,41,Female,atypical angina,130,204,0,lv hypertrophy,172,0,0
5,56,Male,atypical angina,120,236,0,normal,178,0,0
6,62,Female,asymptomatic,140,268,0,lv hypertrophy,160,0,3
7,57,Female,asymptomatic,120,354,0,normal,163,1,0
8,63,Male,asymptomatic,130,254,0,lv hypertrophy,147,0,2
9,53,Male,asymptomatic,140,203,1,lv hypertrophy,155,1,1


In [ ]:
model = BayesianNetwork([('age', 'trestbps'), ('sex', 'fbs'), ('trestbps', 'chol'),
                       ('fbs', 'chol'),('chol','num'),('restecg','num'),
                      ('thalch','exang'),('exang','cp'),('cp','num')])
model

In [ ]:
model.fit(heartDisease, estimator=MaximumLikelihoodEstimator)

In [ ]:
for cpd in model.get_cpds():
    print("CPD of {variable}:".format(variable=cpd.variable))
    print(cpd)

CPD of age:
+---------+-----+
| age(37) | 0.1 |
+---------+-----+
| age(41) | 0.1 |
+---------+-----+
| age(53) | 0.1 |
+---------+-----+
| age(56) | 0.1 |
+---------+-----+
| age(57) | 0.1 |
+---------+-----+
| age(62) | 0.1 |
+---------+-----+
| age(63) | 0.2 |
+---------+-----+
| age(67) | 0.2 |
+---------+-----+
CPD of trestbps:
+---------------+---------+---------+---------+---------+---------+---------+---------+---------+
| age           | age(37) | age(41) | age(53) | age(56) | age(57) | age(62) | age(63) | age(67) |
+---------------+---------+---------+---------+---------+---------+---------+---------+---------+
| trestbps(120) | 0.0     | 0.0     | 0.0     | 1.0     | 1.0     | 0.0     | 0.0     | 0.5     |
+---------------+---------+---------+---------+---------+---------+---------+---------+---------+
| trestbps(130) | 1.0     | 1.0     | 0.0     | 0.0     | 0.0     | 0.0     | 0.5     | 0.0     |
+---------------+---------+---------+---------+---------+---------+---------+

In [ ]:
model.get_independencies()

(chol ⟂ cp, exang, restecg, thalch)
(chol ⟂ exang, sex, thalch, cp, restecg | fbs)
(chol ⟂ cp, exang, restecg | thalch)
(chol ⟂ cp, exang, thalch | restecg)
(chol ⟂ cp, exang, restecg, thalch | age)
(chol ⟂ cp, restecg, thalch | exang)
(chol ⟂ cp, exang, restecg, thalch | sex)
(chol ⟂ exang, thalch, cp, restecg, age | trestbps)
(chol ⟂ exang, restecg, thalch | cp)
(chol ⟂ cp, exang, sex, restecg | thalch, fbs)
(chol ⟂ cp, exang, sex, thalch | restecg, fbs)
(chol ⟂ exang, sex, thalch, cp, restecg | age, fbs)
(chol ⟂ cp, sex, restecg, thalch | exang, fbs)
(chol ⟂ sex | num, fbs)
(chol ⟂ cp, exang, restecg, thalch | sex, fbs)
(chol ⟂ exang, sex, thalch, cp, restecg, age | trestbps, fbs)
(chol ⟂ exang, sex, restecg, thalch | cp, fbs)
(chol ⟂ cp, exang | restecg, thalch)
(chol ⟂ cp, exang, restecg | age, thalch)
(chol ⟂ cp, restecg | exang, thalch)
(chol ⟂ cp, exang, restecg | sex, thalch)
(chol ⟂ cp, exang, restecg, age | trestbps, thalch)
(chol ⟂ exang, restecg | cp, thalch)
(chol ⟂ cp, e

In [ ]:
HeartDisease_infer = VariableElimination(model)
HeartDisease_infer

In [ ]:
a = HeartDisease_infer.query(variables=['num'], evidence={'age': 41})
print(a)

+--------+------------+
| num    |   phi(num) |
+========+============+
| num(0) |     0.2527 |
+--------+------------+
| num(1) |     0.2146 |
+--------+------------+
| num(2) |     0.3266 |
+--------+------------+
| num(3) |     0.2062 |
+--------+------------+


In [ ]:
b = HeartDisease_infer.query(variables=['num'], evidence={'age': 41, 'trestbps': 145})
print(b)

+--------+------------+
| num    |   phi(num) |
+========+============+
| num(0) |     0.2473 |
+--------+------------+
| num(1) |     0.2621 |
+--------+------------+
| num(2) |     0.2621 |
+--------+------------+
| num(3) |     0.2285 |
+--------+------------+


In [ ]:
c = HeartDisease_infer.query(variables=['num'], evidence={'age': 41, 'sex': 'Male'})
print(c)

+--------+------------+
| num    |   phi(num) |
+========+============+
| num(0) |     0.2506 |
+--------+------------+
| num(1) |     0.2205 |
+--------+------------+
| num(2) |     0.3205 |
+--------+------------+
| num(3) |     0.2085 |
+--------+------------+


In [ ]:
d = HeartDisease_infer.query(variables=['num'], evidence={'age': 41, 'sex': 'Male', 'fbs': 1, 'restecg': 'normal', 'exang': 0})
print(d)

+--------+------------+
| num    |   phi(num) |
+========+============+
| num(0) |     0.3125 |
+--------+------------+
| num(1) |     0.2292 |
+--------+------------+
| num(2) |     0.2292 |
+--------+------------+
| num(3) |     0.2292 |
+--------+------------+


In [ ]:
import pickle

In [ ]:
with open('heart_bayesian.pkl','wb') as file:
  pickle.dump(model, file)

In [ ]:
with open('heart_bayesian_VE.pkl','wb') as file:
  pickle.dump(HeartDisease_infer, file)